# Little Fighter 2K
### snippets to upscale and enhance game assets
### follow my progress on my [https://johnnys.news/2024/01/little-fighter-2k](https://johnnys.news/2024/01/little-fighter-2k)

### get a hold to all sprite images

In [4]:
#r "nuget: SkiaSharp, 2.88.6"

using System.IO;
using System.Linq;
using System.Diagnostics;
using SkiaSharp;

var allSpriteFiles = Directory.GetFiles(basePath + @"LittleFighter\sprite", "*.bmp", SearchOption.AllDirectories);
var allDatFiles = Directory.GetFiles(basePath + @"LittleFighter\data", "*.dat", SearchOption.AllDirectories);

var basePath = @"D:\\Dev\\LittleFighter2k\\";

Installed Packages SkiaSharp, 2.88.6

Loading extensions from `C:\Users\joach\.nuget\packages\skiasharp\2.88.6\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

### Encryption / Decryption for .dat files

In [5]:
string key = "odBearBecauseHeIsVeryGoodSiuHungIsAGo";

unsafe string Decrypt(string filepath)
{
    int dec, pass;
    byte[] buffer = File.ReadAllBytes(filepath);
    byte[] decryptedtext = new byte[dec = Math.Max(0, buffer.Length - 123)];
    byte* password = stackalloc byte[pass = key.Length];

    if (pass == 0) return Encoding.ASCII.GetString(buffer);

    for (int i = 0; i < pass; i++)
        password[i] = (byte)key[i];

    fixed (byte* b = buffer, d = decryptedtext)
    {
        for (int i = 0, j = 123; i < dec; i++, j++)
            d[i] = (byte)(b[j] - password[i % pass]);
    }

    return Encoding.ASCII.GetString(decryptedtext);
}

### decrypt .dat files to .txt

In [6]:
allDatFiles.ToList().ForEach(file =>
{
    var text = Decrypt(file);
    var txtFile = Path.GetFileNameWithoutExtension(file) + ".txt";
    if(Directory.Exists("txtdata") == false)
    {
        Directory.CreateDirectory("txtdata");
    }
    File.WriteAllText(Path.Combine("txtdata", txtFile), text);
});

### extract sprites#
extracted files will be placed in the input folder and named as follows:
```
spritesheetfile____row-column.png
```


In [7]:
allDatFiles.ToList().ForEach(file =>
{    
    var txtFile = Path.GetFileNameWithoutExtension(file) + ".txt";
    var lines = File.ReadAllLines(Path.Combine("txtdata", txtFile));

    var files = lines.Where(l => l.StartsWith("file(")).ToList();
    var border = 1.0f;
    for (int i = 0; i < files.Count; i++)
    {
        var fileLine = files[i];
        var fileLineParts = fileLine.Split(" ", StringSplitOptions.RemoveEmptyEntries);
        var fileName = fileLineParts[1];
        var width = int.Parse(fileLineParts[3]);
        var height = int.Parse(fileLineParts[5]);
        var col = int.Parse(fileLineParts[7]);
        var row = int.Parse(fileLineParts[9]);
        var spriteFile = allSpriteFiles.FirstOrDefault(f => f.Contains(fileName));
        if (spriteFile == null)
        {
            Console.WriteLine("Cannot find sprite file for " + fileName);
            continue;
        }
        var sprite = SKBitmap.Decode(spriteFile);
        var spriteWidth = sprite.Width / col;
        var spriteHeight = sprite.Height / row;

        spriteWidth = width;
        spriteHeight = height;
        for (int r = 0; r < row; r++)
        {
            for (int c = 0; c < col; c++)
            {

                var rect = new SKRect(c * spriteWidth + c * border, r * spriteHeight + r * border, (c + 1) * spriteWidth + c * border, (r + 1) * spriteHeight + r * border);
            
                var spriteRect = new SKRect(0, 0, spriteWidth, spriteHeight);
                var spriteBitmap = new SKBitmap(spriteWidth, spriteHeight);
                using (var canvas = new SKCanvas(spriteBitmap))
                {
                    canvas.Clear(SKColors.Black);
                    canvas.DrawBitmap(sprite, rect, new SKRect(0, 0, spriteWidth, spriteHeight));
                }
                var spritePngFile = Path.GetFileNameWithoutExtension(fileName) + "____" + r + "-" + c + ".png";

                if (Directory.Exists("input") == false)
                {
                    Directory.CreateDirectory("input");
                }
                using (var fs = File.OpenWrite(Path.Combine("input", spritePngFile)))
                {
                    spriteBitmap.Encode(SKEncodedImageFormat.Png, 100).SaveTo(fs);
                }
            }
        }
    }

});
Console.WriteLine("Done");

Done


### get other game assets

In [8]:
//get character images and faces
var characters = Directory.EnumerateFiles(@"LittleFighter\sprite", "*.bmp", SearchOption.AllDirectories).Where(f => f.EndsWith("_s.bmp") || f.EndsWith("_f.bmp") || f == "face.bmp" || f == "s.bmp").ToList();
characters.ForEach(character =>
{
   var appendix = character.Contains("template") ? "sprite_template_" : "sprite_";
   File.Copy(character, Path.Combine("input", appendix  + Path.GetFileNameWithoutExtension(Path.GetDirectoryName(character)) + "_" + Path.GetFileName(character).Replace(".bmp", ".png")), true);
});

var bgs = Directory.GetFiles(basePath + @"LittleFighter\bg", "*.bmp", SearchOption.AllDirectories).ToList();
bgs.ForEach(bg =>
{
   var appendix = bg.Contains("template") ? "bg_template_" : "bg_sys_";
   File.Copy(bg, Path.Combine("input", appendix + Path.GetFileNameWithoutExtension(Path.GetDirectoryName(bg)) + "_" + Path.GetFileName(bg).Replace(".bmp", ".png")), true);
});

### upscale with realesrgan-ncnn-vulkan

In [ ]:
#download realesrgan from https://github.com/upscayl/upscayl-ncnn/releases/download/2023.12.06/realesrgan-ncnn-vulkan-2023.12.06-windows.zip
Invoke-WebRequest -Uri "https://github.com/upscayl/upscayl-ncnn/releases/download/2023.12.06/realesrgan-ncnn-vulkan-2023.12.06-windows.zip" -OutFile "realesrgan-ncnn-vulkan-2023.12.06-windows.zip"

#unzip
Expand-Archive -Path "realesrgan-ncnn-vulkan-2023.12.06-windows.zip" -DestinationPath "realesrgan-ncnn-vulkan"

# get model from https://github.com/upscayl/custom-models and put it in the models folder

# upscale with different models
#./realesrgan-ncnn-vulkan/realesrgan-ncnn-vulkan.exe -i input -o ./output/general -m ./realesrgan-ncnn-vulkan-2023.12.06-windows/models -n RealESRGAN_General_x4_v3 -v
#./realesrgan-ncnn-vulkan/realesrgan-ncnn-vulkan.exe -i input -o ./output/x4plus -m ./realesrgan-ncnn-vulkan-2023.12.06-windows/models -n realesrgan-x4plus -v
#./realesrgan-ncnn-vulkan/realesrgan-ncnn-vulkan.exe -i input -o ./output/x4plus-anime -m ./realesrgan-ncnn-vulkan-2023.12.06-windows/models -n realesrgan-x4plus-anime -v
#./realesrgan-ncnn-vulkan/realesrgan-ncnn-vulkan.exe -i input -o ./output/remacri -m ./realesrgan-ncnn-vulkan-2023.12.06-windows/models -n remacri -v
#./realesrgan-ncnn-vulkan/realesrgan-ncnn-vulkan.exe -i input -o ./output/ultrasharp -m ./realesrgan-ncnn-vulkan-2023.12.06-windows/models -n ultrasharp -v
#./realesrgan-ncnn-vulkan/realesrgan-ncnn-vulkan.exe -i input -o ./output/ultramix-balanced -m ./realesrgan-ncnn-vulkan-2023.12.06-windows/models -n ultramix_balanced -v

# test for double upscale
#./realesrgan-ncnn-vulkan/realesrgan-ncnn-vulkan.exe -i input -o ./temp -m ./realesrgan-ncnn-vulkan-2023.12.06-windows/models -n ultrasharp -v
#./realesrgan-ncnn-vulkan/realesrgan-ncnn-vulkan.exe -i temp -o ./output -m ./realesrgan-ncnn-vulkan-2023.12.06-windows/models -n ultrasharp -v

mkdir output

# upscale sprites
./realesrgan-ncnn-vulkan/realesrgan-ncnn-vulkan.exe -i input -o output -m ./realesrgan-ncnn-vulkan-2023.12.06-windows/models -n ultrasharp -v


### build valid images collection (ignore completely black images and #010000)

In [ ]:
var validImages = new List<string>();

var almostBlack = SKColor.Parse("#010000");
foreach (var file in validFiles)
{
    using var bitmap = SKBitmap.Decode(file);
    var isBlack = true;
    foreach (var pixel in bitmap.Pixels)
    {
        if (pixel == SKColors.Black || pixel == almostBlack)
        {
            continue;
        }
        isBlack = false;
    }
    if (!isBlack)
    {
        validImages.Add(file);
    }
}

### use Replicate https://replicate.com/lucataco/demofusion-enhance

In [3]:
var validFiles = Directory.GetFiles(basePath + "valid", "*.png", SearchOption.AllDirectories).ToList();
var existing = Directory.GetFiles(basePath + "replicate\\output", "*.png", SearchOption.AllDirectories).ToList();
var predictions = new List<(string filename, string id)>();
var output = new List<(string result, string file)>();

validFiles = validFiles.Where(x => !existing.Any(y => Path.GetFileName(y) == Path.GetFileName(x))).ToList();

validFiles = validFiles.Take(30).ToList();


foreach (var file in validFiles)
{
    var client = new HttpClient();
    var url = "https://api.replicate.com/v1/predictions";
    var imageUrl = "image uploaded somehwere";
    var json = @"{
    ""version"": ""5bcfe11066c820e8c08232c6efa3c8a7ab2cd667ad136ca173633f352170691e"",
    ""input"": {
      ""seed"": 123,
      ""image"": """ + imageUrl + @""",
      ""scale"": 2,
      ""sigma"": 0.8,
      ""prompt"": ""videogame rendered human+++ martial arts fighter, fine details, texture+++, rendering, 3D, fighting pose"",
      ""stride"": 64,
      ""auto_prompt"": true,
      ""multi_decoder"": false,
      ""cosine_scale_1"": 3,
      ""cosine_scale_2"": 1,
      ""cosine_scale_3"": 1,
      ""guidance_scale"": 18,
      ""negative_prompt"": ""blurry, ugly, duplicate, poorly drawn, deformed, mosaic, anime, pixelated, cartoon, low resolution, low quality, low res, low resolution, low"",
      ""view_batch_size"": 16,
      ""num_inference_steps"": 40
    }
}";
    var data = new StringContent(json, Encoding.UTF8, "application/json");
    client.DefaultRequestHeaders.Add("Authorization", "Token YOURTOKEN");
    var response = await client.PostAsync(url, data);
    var result = await response.Content.ReadAsStringAsync();

    JsonNode prediction = JsonSerializer.Deserialize<JsonNode>(result);
    var id = prediction["id"].GetValue<string>();

    predictions.Add((file, id));
}

await Task.Delay(100000);

while (output.Count < predictions.Count)
{
    foreach (var prediction in predictions)
    {
        try
        {
            if (output.Any(x => x.file == prediction.filename))
            {
                continue;
            }
            var client = new HttpClient();
            var url = "https://api.replicate.com/v1/predictions";
            client.DefaultRequestHeaders.Add("Authorization", "Token YOURTOKEN");
            var response = await client.GetAsync(url + "/" + prediction.id);
            var json = await response.Content.ReadAsStringAsync();
            var result = JsonSerializer.Deserialize<JsonNode>(json);
            var status = result["status"].GetValue<string>();
            var outputnode = result["output"].GetValue<string>();
            if (status == "succeeded")
            {
                output.Add((outputnode, prediction.filename));
            }
        }
        catch (Exception ex)
        {
            Console.WriteLine(ex.Message);
        }
        await Task.Delay(10000);

    }

}

// save output to file
var outputprint = output.Select(x => x.result + " " + x.file).ToList();
File.WriteAllText(basePath + "replicate\\output\\output.json", JsonSerializer.Serialize(outputprint));

foreach (var item in output)
{
    var client = new HttpClient();
    var url = item.result;
    var response = await client.GetAsync(url);
    var bytes = await response.Content.ReadAsByteArrayAsync();
    var path = basePath + "replicate\\output";
    File.WriteAllBytes(Path.Combine(path, Path.GetFileName(item.file)), bytes);
}


System.Console.WriteLine("Done");

Error: (11,5): error CS1002: ; expected

### combined upscaled sprites to spritesheets

In [ ]:
var upscaled = Directory.GetFiles(basePath + @"enhanced", "*.png", SearchOption.AllDirectories).Where(f => f.Contains("____")).ToList();

var resize = false;
var upscaleFactor = 4;
if(!resize)
{
    upscaleFactor = 1;
}
var border = 1.0f;
allDatFiles.ToList().ForEach(file =>
{   
    var txtFile = Path.GetFileNameWithoutExtension(file) + ".txt";
    var lines = File.ReadAllLines(Path.Combine("txtdata", txtFile));
    var files = lines.Where(l => l.StartsWith("file(")).ToList();
    for (int i = 0; i < files.Count; i++)
    {
        var fileLine = files[i];
        var fileLineParts = fileLine.Split(" ", StringSplitOptions.RemoveEmptyEntries);
        var fileName = fileLineParts[1];
        var width = int.Parse(fileLineParts[3]) * upscaleFactor;
        var height = int.Parse(fileLineParts[5]) * upscaleFactor;
        var col = int.Parse(fileLineParts[7]);
        var row = int.Parse(fileLineParts[9]);
        var spriteFile = allSpriteFiles.FirstOrDefault(f => f.Contains(fileName));
        if (spriteFile == null)
        {
            Console.WriteLine("Cannot find sprite file for " + fileName);
            continue;
        }

        var original = SKBitmap.Decode(spriteFile);
        var result = new SKBitmap(original.Width * upscaleFactor, original.Height * upscaleFactor);
        using var canvas = new SKCanvas(result);
        canvas.Clear(SKColors.Black);
        for (int r = 0; r < row; r++)
        {
            for (int c = 0; c < col; c++)
            {
                var upscaledFile = upscaled.FirstOrDefault(f => f == Path.Combine("enhanced", Path.GetFileNameWithoutExtension(fileName) + "____" + r + "-" + c + ".png"));
                var sprite = upscaled.FirstOrDefault(f => f == upscaledFile);
                var rect = new SKRect(c * width + c * border, r * height + r * border, (c + 1) * width + c * border, (r + 1) * height + r * border);

                if (sprite != null)
                {
                    if(resize)
                    {
                        var spriteBitmap = SKBitmap.Decode(sprite);
                        var scaledBitmap = new SKBitmap(spriteBitmap.Width / upscaleFactor, spriteBitmap.Height / upscaleFactor);
                        var scaled = spriteBitmap.ScalePixels(scaledBitmap, SKFilterQuality.High);
                        canvas.DrawBitmap(scaledBitmap, rect);
                    }
                    else
                    {
                        var spriteBitmap = SKBitmap.Decode(sprite);
                        canvas.DrawBitmap(spriteBitmap, new SKRect(0, 0, width, height), rect);
                    }
                }
                else
                {
                    canvas.DrawRect(rect, new SKPaint() { Color = SKColors.Black });
                }
            }
        }


        Directory.CreateDirectory("result");
        Directory.CreateDirectory(Path.Combine("result", "sprite"));
        Directory.CreateDirectory(Path.Combine("result", "sprite", "sys"));
        Directory.CreateDirectory(Path.Combine("result", "sprite", "template"));

        var resultPath = file.Contains("template") ? Path.Combine("result", "sprite", "template", Path.GetFileNameWithoutExtension(fileName) + ".png") : Path.Combine("result", "sprite", "sys", Path.GetFileNameWithoutExtension(fileName) + ".png");

        using (var fs = File.OpenWrite(Path.Combine(resultPath)))
        {
            result.Encode(SKEncodedImageFormat.Png, 100).SaveTo(fs);
        }
    }
});

### replace original sprites with upscaled sprites

In [10]:
var outputSprites = Directory.GetFiles(basePath + @"output", "*.bmp", SearchOption.AllDirectories);
allSpriteFiles.ToList().ForEach(file =>
{
    var filename = Path.GetFileNameWithoutExtension(file);
    var outputSprite = outputSprites.FirstOrDefault(f => f.Contains(filename));
    
    if(outputSprite != null)
    {
        File.Copy(outputSprite, file, true);
    }
});

to be continued...